# Práctico 5: Generación de Variables Aleatorias Continuas


In [81]:
import random as rnd
import numpy as np
from math import *
from scipy.stats import chi2
from tabulate import tabulate

## Utilidades de unidades anteriores


In [3]:
def normal_distribution(m: float, s: float, sz: int = 1, method: str = "rejection") -> list[float]:
    """
    Generates random numbers from a normal distribution.

    Parameters:
    m: float
        The mean of the normal distribution.
    s: float
        The standard deviation of the normal distribution.
    sz: int
        The number of random numbers to generate.
    method: str
        The method to use for generating random numbers. Can be "rejection", "polar", "box-muller" or "uniform_ratios".
        Default is "rejection".

    Returns:
    list[float]
        A list of random numbers from the normal distribution.
    """
    def rejection_method() -> float:
        while True:
            u, y = -np.log(rnd.random()), -np.log(rnd.random())
            if y >= (u - 1) ** 2 / 2:
                return (u if rnd.random() < 0.5 else -u)

    def polar_method() -> list[float]:
        r2 = -2 * np.log(1 - rnd.random())
        theta = 2 * np.pi * rnd.random()
        x, y = np.sqrt(r2) * np.cos(theta), np.sqrt(r2) * np.sin(theta)
        return [x, y]

    def box_muller_method() -> list[float]:
        while True:
            v1, v2 = 2 * rnd.random() - 1, 2 * rnd.random() - 1
            if v1 ** 2 + v2 ** 2 <= 1:
                s = v1 ** 2 + v2 ** 2
                x, y = v1 * np.sqrt(-2 * np.log(s) / s), v2 * \
                    np.sqrt(-2 * np.log(s) / s)
                return [x, y]

    def uniform_ratios_method() -> float:
        c = 4 * np.exp(-0.5) / np.sqrt(2.0)
        while True:
            u, y = rnd.random(), 1 - rnd.random()
            z = c * (u - 0.5) / y
            if z ** 2 / 4 <= -np.log(y):
                return z

    match method:
        case "polar":
            r = []
            while len(r) < sz:
                r += [m + x * s for x in polar_method()]
            return r[:sz]
        case "box-muller":
            r = []
            while len(r) < sz:
                r += [m + x * s for x in box_muller_method()]
            return r[:sz]
        case "uniform_ratios":
            return [m + uniform_ratios_method() * s for _ in range(sz)]
        case "rejection":
            return [m + rejection_method() * s for _ in range(sz)]
        case _:
            raise ValueError(
                "Invalid method. Please use 'rejection', 'polar', 'box-muller' or 'uniform_ratios'.")

In [87]:
def binomial_distribution(n: int, p: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a binomial distribution.

    Parameters:
    n: int - Number of trials.
    p: float - Probability of success.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(n: int, p: float) -> int:
        c = p / (1 - p)
        prob = (1 - p)**n
        F = prob
        U = rnd.random()

        for j in range(1, int(n * p) + 1):
            prob *= c * (n - j + 1) / j
            F += prob
        if U >= F:
            j = int(n * p) + 1
            while U >= F:
                prob *= c * (n - j + 1) / j
                F += prob
                j += 1
            return j - 1
        else:
            j = int(n * p)
            while U < F:
                F -= prob
                prob *= j / (c * (n - j + 1))
                j -= 1
            return j + 1

    if p > 0.5:
        return [n - generate(n, 1 - p) for _ in range(sz)]
    else:
        return [generate(n, p) for _ in range(sz)]

In [137]:
def inverse_transform_method(G: callable) -> float:
    """
    Inverse Transform Method for generating random numbers from a given distribution.

    Parameters:
    G: callable
        The inverse of the cumulative distribution function of the distribution from which we want to generate random numbers.

    Returns:
    float
        A random number from the distribution.
    """
    U = rnd.random()
    return G(U)


def gamma_distribution(a: int, b: float, sz: int = 1) -> list[float]:
    """
    Generates random numbers from a gamma distribution with shape parameter ('a') as integer.
    I.e., a Erlang distribution.

    Parameters:
    a: int
        The shape parameter of the gamma distribution.
    b: float
        The rate parameter of the gamma distribution.
    sz: int
        The number of random numbers to generate.

    Returns:
    list[float]
        A list of random numbers from the gamma distribution.
    """
    def generate_gamma(a: int, b: float) -> float:
        u = 1
        for _ in range(a):
            u *= 1 - rnd.random()
        assert u > 0, "Invalid value generated for u"
        return -np.log(u) * b

    return [generate_gamma(a, b) for _ in range(sz)]


def exponential_distribution(l: float, sz: int = 1, gamma_method: bool = True) -> list[float]:
    """
    Generates random numbers from an exponential distribution.

    Parameters:
    l: float
        The rate parameter of the exponential distribution.
    sz: int
        The number of random numbers to generate.
    gamma_method: bool
        If True, uses the gamma distribution method to generate random numbers. Otherwise, uses the inverse transform method.
        (The gamma method is more efficient for generating multiple random numbers but if the SZ is big, -log(u) can be very close to 0 and cause numerical issues.)

    Returns:
    list[float]
        A list of random numbers from the exponential distribution.
    """
    if sz == 1 or not gamma_method:
        return [inverse_transform_method(lambda u: -np.log(1 - u) / l) for _ in range(sz)]

    # Method using a Gamma distribution.
    t = gamma_distribution(sz, 1/l)[0]
    u = sorted([rnd.random() for _ in range(sz - 1)] + [0, 1])
    return [(u[i] - u[i-1]) * t for i in range(1, len(u))]

### De esta unidad


## Análisis Estadístico de Datos Simulados


### Utilidades


In [27]:
def sample_mean(data: list[float]) -> float:
    """
    Calculates the sample mean of a list of numbers.

    Parameters:
    data: list[float]
        The list of numbers.

    Returns:
    float
        The sample mean of the list of numbers.
    """
    return sum(data) / len(data)


def sample_variance(data: list[float]) -> float:
    """
    Calculates the sample variance of a list of numbers.

    Parameters:
    data: list[float]
        The list of numbers.

    Returns:
    float
        The sample variance of the list of numbers.
    """
    m = sample_mean(data)
    return sum((x - m) ** 2 for x in data) / (len(data) - 1)

In [9]:
def calculate_principal_estimators(generator: callable = rnd.random, n: int = 1, d: float = 0.1):
    """
    Calculate the principal estimators (mean and variance) of the given generator.

    Parameters:
    - generator (callable): A random number generator.
    - n (int): The number of samples to generate.
    - d (float): The limit for standard deviation.

    Returns:
    - mean (float): The mean of the samples.
    - variance (float): The variance of the samples.
    - n (int): The number of samples generated.
    """

    mean = generator()
    s2 = 0
    i = 1
    while i <= n or np.sqrt(s2/i) > d:
        i += 1
        X = generator()

        prev_mean = mean
        mean = mean + (X - mean) / i
        s2 = s2 * (1 - 1/(i-1)) + i * (mean - prev_mean)**2

    return mean, s2, i

### Ejercicio 1


In [16]:
n = 100
d = 0.1


def generator(): return normal_distribution(0, 1, 1)[0]


mean, variance, n = calculate_principal_estimators(generator, n, d)

print(f"Number of samples: {n}")
print(f"Mean: {mean}")
print(f"Variance: {variance}")

Number of samples: 123
Mean: -0.009755856823913332
Variance: 1.2126562072241944


### Ejercicio 2


In [20]:
def generator_i():
    U = rnd.random()
    return np.exp(U) / np.sqrt(2 * U)


def generator_ii():
    def f(x): return x**2 * np.exp(-x**2)
    U = rnd.random()
    return f(1/U - 1) / U**2


n = 100
d = 0.01
mean_i, variance_i, n_i = calculate_principal_estimators(generator_i, n, d)
mean_ii, variance_ii, n_ii = calculate_principal_estimators(generator_ii, n, d)

data = [["i", mean_i, n_i],
        ["ii", 2*mean_ii, n_ii]]
headers = ["Exercise", "Estimation of the integral", "Number of samples"]
print(tabulate(data, headers, tablefmt="fancy_grid"))

╒════════════╤══════════════════════════════╤═════════════════════╕
│ Exercise   │   Estimation of the integral │   Number of samples │
╞════════════╪══════════════════════════════╪═════════════════════╡
│ i          │                     2.05813  │               35236 │
├────────────┼──────────────────────────────┼─────────────────────┤
│ ii         │                     0.896966 │                3185 │
╘════════════╧══════════════════════════════╧═════════════════════╛


### Ejercicio 3


In [31]:
def generator_i():
    U = rnd.random()
    return np.sin(np.pi*(U + 1))/(U + 1)


def generator_ii():
    def f(x): return 3/(3 + x**4)
    U = rnd.random()
    return f(1/U - 1) / U**2


z_alpha_2 = 1.96
d = 0.001 / (2 * z_alpha_2)
n = 100

data_i = calculate_principal_estimators(generator_i, n, d)
data_ii = calculate_principal_estimators(generator_ii, n, d)

# Generate answer


def g(gen, x):
    data = [gen() for _ in range(x)]
    return [sample_mean(data), sample_variance(data), x]


nro_sim = [1000, 5000, 7000]
ans_i = list(map(lambda x: g(generator_i, x), nro_sim)) + [data_i]
ans_ii = list(map(lambda x: g(generator_ii, x), nro_sim)) + [data_ii]


def confidence_interval(z_alpha_2, mean, var, n):
    B = np.sqrt(var / n) * z_alpha_2
    return mean - B, mean + B


ans_i = list(map(lambda x: [x[0], x[1], x[2], confidence_interval(
    z_alpha_2, x[0], x[1], x[2])], ans_i))
ans_ii = list(map(lambda x: [x[0], x[1], x[2], confidence_interval(
    z_alpha_2, x[0], x[1], x[2])], ans_ii))

headers = ["Estimation of the integral", "Variance",
           "Number of samples", "Confidence interval"]
print('For the first integral:')
print(tabulate(ans_i, headers, tablefmt="fancy_grid"))
print('For the second integral:')
print(tabulate(ans_ii, headers, tablefmt="fancy_grid"))

For the first integral:
╒══════════════════════════════╤════════════╤═════════════════════╤══════════════════════════════════════════════╕
│   Estimation of the integral │   Variance │   Number of samples │ Confidence interval                          │
╞══════════════════════════════╪════════════╪═════════════════════╪══════════════════════════════════════════════╡
│                    -0.437815 │  0.0441577 │                1000 │ (-0.45083924771680595, -0.42479035720043623) │
├──────────────────────────────┼────────────┼─────────────────────┼──────────────────────────────────────────────┤
│                    -0.430717 │  0.044912  │                5000 │ (-0.43659084236042267, -0.4248423442371536)  │
├──────────────────────────────┼────────────┼─────────────────────┼──────────────────────────────────────────────┤
│                    -0.432222 │  0.0440323 │                7000 │ (-0.4371381514306426, -0.4273065852504645)   │
├──────────────────────────────┼────────────┼───────────

### Ejercicio 4


In [49]:
def generate_data():
    s, i = 0, 0
    while s < 1:
        s += rnd.random()
        i += 1
    return i


def e_estimator(n):
    sample = [generate_data() for _ in range(n)]
    return sample_mean(sample)


# Item B
n = 1000
data = [e_estimator(n) for _ in range(n)]
var = sample_variance(data)
print(f'Item B: Variance of the estimator: {var}')

# Item C
n = 1000
a = 0.05
l = 0.025
z_alpha_2 = 1.96
d = l / (2 * z_alpha_2)

mean, variance, n = calculate_principal_estimators(generate_data, n, d)
print('Item C')
headers = ['Real value of e', 'Estimation of e',
           'Variance', 'Number of samples']
data = [np.e, mean, variance, n]
print(tabulate([data], headers, tablefmt="fancy_grid"))

Item B: Variance of the estimator: 0.000741342298298299
Item C
╒═══════════════════╤═══════════════════╤════════════╤═════════════════════╕
│   Real value of e │   Estimation of e │   Variance │   Number of samples │
╞═══════════════════╪═══════════════════╪════════════╪═════════════════════╡
│           2.71828 │            2.7202 │   0.771703 │               18974 │
╘═══════════════════╧═══════════════════╧════════════╧═════════════════════╛


### Ejercicio 5


In [69]:
def generar_M():
    s, i = 0, 1
    U = rnd.random()
    while s < U:
        s = U
        U = rnd.random()
        i += 1
    return i


# Item C
n = 1000
d = 0.01
mean, variance, n = calculate_principal_estimators(generar_M, n, d)
print('Item C')
headers = ['Real value of e', 'Estimation of e',
           'Variance', 'Number of samples']
data = [np.e, mean, variance, n]
print(tabulate([data], headers, tablefmt="fancy_grid"))

# Item D
z_alpha_2 = 1.96
l = 0.1
d = l / (2 * z_alpha_2)
n = 1000

mean, variance, n = calculate_principal_estimators(generar_M, n, d)


def confidence_interval(z_alpha_2, mean, var, n):
    B = np.sqrt(var / n) * z_alpha_2
    return mean - B, mean + B


headers = ['Estimation of e', 'Variance',
           'Number of samples', 'Confidence interval']
data = [mean, variance, n, confidence_interval(z_alpha_2, mean, variance, n)]
print('Item D')
print(tabulate([data], headers, tablefmt="fancy_grid"))

Item C
╒═══════════════════╤═══════════════════╤════════════╤═════════════════════╕
│   Real value of e │   Estimation of e │   Variance │   Number of samples │
╞═══════════════════╪═══════════════════╪════════════╪═════════════════════╡
│           2.71828 │           2.72676 │   0.781981 │                7821 │
╘═══════════════════╧═══════════════════╧════════════╧═════════════════════╛
Item D
╒═══════════════════╤════════════╤═════════════════════╤══════════════════════════════════════════╕
│   Estimation of e │   Variance │   Number of samples │ Confidence interval                      │
╞═══════════════════╪════════════╪═════════════════════╪══════════════════════════════════════════╡
│           2.73032 │   0.718872 │                1105 │ (2.6803247279204987, 2.7803087562423974) │
╘═══════════════════╧════════════╧═════════════════════╧══════════════════════════════════════════╛


### Ejercicio 6


In [78]:
def generate_P():
    X = rnd.random()
    Y = rnd.random()
    return 1 if X**2 + Y**2 <= 1 else 0


l = 0.1
z_alpha_2 = 1.96
d = l / (8 * z_alpha_2)
n = 1000

mean, variance, n = calculate_principal_estimators(generate_P, n, d)

headers = ['Real value of pi', 'Estimation of pi',
           'Variance', 'Number of samples']
data = [np.pi, mean * 4, variance, n]
print('Item A')
print(tabulate([data], headers, tablefmt="fancy_grid"))

Item A
╒════════════════════╤════════════════════╤════════════╤═════════════════════╕
│   Real value of pi │   Estimation of pi │   Variance │   Number of samples │
╞════════════════════╪════════════════════╪════════════╪═════════════════════╡
│            3.14159 │            3.13969 │    0.16886 │                4152 │
╘════════════════════╧════════════════════╧════════════╧═════════════════════╛


## Técnicas de Validación Estadística


### Utilidades


#### Datos discretos - Test chi-cuadrado de Pearson


In [111]:
def pearson_estimator(N: list[int], p: list[float]) -> float:
    """
    Calculates the Pearson estimator of a list of numbers.

    Parameters:
    N: list[int]
        The list of frequencies.
    p: list[float]
        The list of probabilities.

    Returns:
    float
        The Pearson estimator of the list of numbers.
    """
    assert len(N) == len(p)

    n = sum(N)
    return sum((N[i] - n * p[i]) ** 2 / (n * p[i]) for i in range(len(N)))


def pearson_p_value(t: float, df: int) -> float:
    """
    Calculates the p-value of the Pearson estimator.

    Parameters:
    t: float
        The value of the Pearson estimator.
    df: int
        The degrees of freedom.

    Returns:
    float
        The p-value of the Pearson estimator.
    """
    return 1.0 - chi2.cdf(t, df)


def pearson_p_value_sim(t: float, n_sim: int, p: list[float], n: int):
    """
    Calculates the p-value of the Pearson estimator using simulation.

    Parameters:
    t: float
        The value of the Pearson estimator.
    n_sim: int
        The number of simulations to run.
    p: list[float]
        The list of probabilities.
    n: int
        The size of the original sample.

    Returns:
    float
        The p-value of the Pearson estimator.
    """
    def create_observation_freq(n: int, p: list[float]):
        N = np.zeros(len(p))
        N[0] = binomial_distribution(n, p[0])[0]
        for i in range(1, len(p) - 1):
            N[i] = binomial_distribution(
                n - sum(N[:i]), p[i]/(1 - sum(p[:i])))[0]
        N[-1] = n - sum(N[:-1])
        return np.array(N, dtype=int)

    p_value = 0
    for _ in range(n_sim):
        N = create_observation_freq(n, p)
        t_sim = pearson_estimator(N, p)
        if t_sim >= t:
            p_value += 1
    return p_value / n_sim

#### Datos continuos - Test de Kolmogorov-Smirnov


In [103]:
def kolmogorov_estimator(sample: list[float], F: callable):
    """
    Calculates the Kolmogorov estimator of a list of numbers.

    Parameters:
    sample: list[float]
        The list of numbers.
    F: callable
        The cumulative distribution function.

    Returns:
    float
        The Kolmogorov estimator of the list of numbers.
    """

    n = len(sample)
    s_sample = sorted(sample)

    D = 0
    for i in range(n):
        D = max((i + 1) / n - F(s_sample[i]), F(s_sample[i]) - i / n, D)

    return D


def kolmogorov_p_value_sim(d: float, n_sim: int, n: int):
    """
    Calculates the p-value of the Kolmogorov estimator using simulation.

    Parameters:
    d: float
        The value of the Kolmogorov estimator.
    n_sim: int
        The number of simulations to run.
    n: int
        The size of the original sample.

    Returns:
    float
        The p-value of the Kolmogorov estimator.
    """
    p_value = 0
    for _ in range(n_sim):
        sample = [rnd.random() for _ in range(n)]
        D = kolmogorov_estimator(sample, lambda x: x)
        if D >= d:
            p_value += 1

    return p_value / n_sim

### Ejercicio 7


In [97]:
# DATA
p = [1/4, 1/2, 1/4]
N = [141, 291, 132]

# Item A
t = pearson_estimator(N, p)
p_value = pearson_p_value(t, len(p) - 1)
print(f'p-value: {p_value}')

# Item B
n_sim = 1000
p_value_sim = pearson_p_value_sim(t, n_sim, p, sum(N))
print(f'p-value (simulation): {p_value_sim}')

p-value: 0.6499557054800363
p-value (simulation): 0.639


### Ejercicio 8


In [98]:
p = [1/6] * 6
N = [158, 172, 164, 181, 160, 165]

# Item a
t = pearson_estimator(N, p)
p_value = pearson_p_value(t, len(p) - 1)
print(f'p-value: {p_value}')

# Item b
n_sim = 1000
p_value_sim = pearson_p_value_sim(t, n_sim, p, sum(N))
print(f'p-value (simulation): {p_value_sim}')

p-value: 0.8237195392577814
p-value (simulation): 0.823


### Ejercicio 9


In [105]:
def uniform_F(x: float) -> float:
    return x if 0 <= x <= 1 else 0


numbers = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]
D = kolmogorov_estimator(numbers, uniform_F)
print(f'Kolmogorov estimator: {D}')

n_sim = 1000
n = len(numbers)
p_value_sim = kolmogorov_p_value_sim(D, n_sim, n)
print(f'p-value (simulation): {p_value_sim}')

Kolmogorov estimator: 0.24
p-value (simulation): 0.54


### Ejercicio 10


In [106]:
def exponential_F(x: float) -> float:
    if x < 0:
        return 0
    return 1 - np.exp(-1/50 * x)


numbers = [86, 133, 75, 22, 11, 144, 78, 122, 8, 146, 33, 41, 99]
D = kolmogorov_estimator(numbers, exponential_F)
print(f'Kolmogorov estimator: {D}')

n_sim = 1000
n = len(numbers)
p_value_sim = kolmogorov_p_value_sim(D, n_sim, n)
print(f'p-value (simulation): {p_value_sim}')

Kolmogorov estimator: 0.3922544552361856
p-value (simulation): 0.024


### Ejercicio 11


In [134]:
N = 8


def binomial_probability(x, p):
    if x < 0 or x > N:
        return 0
    return comb(N, x) * p**x * (1 - p)**(N - x)


# Data
numbers = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
n = len(numbers)
p_estimator = sample_mean(numbers) / N
p = [binomial_probability(x, p_estimator) for x in range(N+1)]
N_obs = [numbers.count(x) for x in range(N+1)]

# With Pearson estimator
t = pearson_estimator(N_obs, p)
p_value = pearson_p_value(t, len(p) - 2)
print(f'With Pearson estimator, p-value: {p_value}')

# With Simulation
n_sim = 1000
p_value_sim = 0

for _ in range(n_sim):
    sample_sim = binomial_distribution(N, p_estimator, n)
    p_sim_estimator = sample_mean(sample_sim) / N
    N_obs_sim = [sample_sim.count(x) for x in range(N+1)]
    t_sim = pearson_estimator(N_obs_sim, p)

    if t_sim >= t:
        p_value_sim += 1

p_value_sim /= n_sim

print(f'With Simulation, p-value: {p_value_sim}')

With Pearson estimator, p-value: 5.0279943204278865e-05
With Simulation, p-value: 0.009


### Ejercicio 12


In [136]:
p = [0.31, 0.22, 0.12, 0.10, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01]
N = [188, 138, 87, 65, 48, 32, 30, 34, 13, 2]

# With Pearson estimator
t = pearson_estimator(N, p)
p_value = pearson_p_value(t, len(p) - 1)
print(f'With Pearson estimator, p-value: {p_value}')

# With Simulation
n_sim = 1000
p_value_sim = pearson_p_value_sim(t, n_sim, p, sum(N))
print(f'With Simulation, p-value: {p_value_sim}')

With Pearson estimator, p-value: 0.36605389988682613
With Simulation, p-value: 0.374


### Ejercicio 13


In [140]:
def exponential_F(x: float) -> float:
    if x < 0:
        return 0
    return 1 - np.exp(-x)


numbers = exponential_distribution(1, 30)
D = kolmogorov_estimator(numbers, exponential_F)
print(f'Kolmogorov estimator: {D}')

n_sim = 1000
n = len(numbers)
p_value_sim = kolmogorov_p_value_sim(D, n_sim, n)
print(f'p-value (simulation): {p_value_sim}')

Kolmogorov estimator: 0.16365846955888136
p-value (simulation): 0.392


### Ejercicio 14


In [157]:
def t_student_distribution(df):
    x = rnd.gauss(0, 1)
    y = 2*rnd.gammavariate(0.5*df, 2)
    return x/(sqrt(y/df))


def normal_cumulative_z(x):
    return erf(x/sqrt(2))/2+0.5


headers = ['n', 'estimator', 'p-value']
table_data = []

sizes = [10, 20, 100, 1000]
for n in sizes:
    data = [t_student_distribution(11) for _ in range(n)]
    D = kolmogorov_estimator(data, normal_cumulative_z)
    p_value = kolmogorov_p_value_sim(D, 1000, n)
    table_data.append([n, D, p_value])

print(tabulate(table_data, headers, tablefmt="fancy_grid"))

╒══════╤═════════════╤═══════════╕
│    n │   estimator │   p-value │
╞══════╪═════════════╪═══════════╡
│   10 │   0.254871  │     0.457 │
├──────┼─────────────┼───────────┤
│   20 │   0.158162  │     0.632 │
├──────┼─────────────┼───────────┤
│  100 │   0.172572  │     0.003 │
├──────┼─────────────┼───────────┤
│ 1000 │   0.0761654 │     0     │
╘══════╧═════════════╧═══════════╛


### Ejercicio 15


In [160]:
def exponential_F(l: float, x: float):
    if x < 0:
        return 0
    return 1 - np.exp(-l * x)


numbers = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3,
           10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]
l = 1/sample_mean(numbers)
D = kolmogorov_estimator(numbers, lambda x: exponential_F(l, x))
print(f'Kolmogorov estimator: {D}')

n_sim = 1000
n = len(numbers)
p_value_sim = 0
for _ in range(n_sim):
    sample = exponential_distribution(l, n)
    l_sim = 1/sample_mean(sample)
    D_sim = kolmogorov_estimator(sample, lambda x: exponential_F(l_sim, x))
    if D_sim >= D:
        p_value_sim += 1
p_value_sim /= n_sim
print(f'p-value (simulation): {p_value_sim}')

Kolmogorov estimator: 0.26949936321059237
p-value (simulation): 0.045


### Ejercicio 16


In [164]:
def normal_cumulative(x: float, m: float, s: float):
    return normal_cumulative_z((x - m) / s)


numbers = [91.9, 97.8, 111.4, 122.3, 105.4,
           95.0, 103.8, 99.6, 96.6, 119.3, 104.8, 101.7]
n = len(numbers)
m, s = sample_mean(numbers), sqrt(sample_variance(numbers))

D = kolmogorov_estimator(numbers, lambda x: normal_cumulative(x, m, s))
print(f'Kolmogorov estimator: {D}')

n_sim = 1000
p_value_sim = 0
for _ in range(n_sim):
    sample = normal_distribution(m, s, n)
    m_sim, s_sim = sample_mean(sample), sqrt(sample_variance(sample))
    D_sim = kolmogorov_estimator(
        sample, lambda x: normal_cumulative(x, m_sim, s_sim))
    if D_sim >= D:
        p_value_sim += 1
p_value_sim /= n_sim
print(f'p-value (simulation): {p_value_sim}')

Kolmogorov estimator: 0.19638944697995597
p-value (simulation): 0.206
